# Functions

## This script contains the following

* Imports libraries and initializes figure parameters
* Imports functions

## The following functions are included:

* UFL_to_csc
* UFL_to_csc_nobcs
* POD
* POD2
* POD_all
* POD_M1
* plot_first_n_svalues
* map_nodal_to_vertex
* block_vector_to_nodal
* U_to_figure
* nodal_to_figure
* split_data
* plot_last_epochs
* relative_error
* my_2norm
* get_inputs
* get_outputs
* input_to_output_model_nonorm
* find_initial_condition
* newdescent
* newton_nonorm
* Neural_Network
* Neural_Network_small
* train_nn
* BFGS_Fu
* BFGS_g
* BFGS_grad_g
* BFGS_line_search
* BFGS
* Fu
* g
* grad_g
* line_search
* standardize
* normalize
* Monte_Carlo
* f_normal
* uni_dist

In [1]:
# --------------------- Import libraries

from dolfin import *
import numpy as np
import scipy.sparse as sps

import matplotlib.pyplot as plt
import random
import time

import os
import torch
from re import X

import torch.nn as nn
import copy
import logging

import math
import pickle

from mpl_toolkits.axes_grid1 import AxesGrid

logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
set_log_level(30) # Level 30: WARNING

params = {
        'axes.labelsize': 21,
        'font.size': 21,
        'legend.fontsize': 15,
        'xtick.labelsize': 15,
        'ytick.labelsize': 15,
        'text.usetex': False,
        'axes.linewidth': 2,
        'xtick.major.width': 2,
        'ytick.major.width': 2,
        'xtick.major.size': 2,
        'ytick.major.size': 2,
    }

plt.rcParams.update(params)

# --------------------- End import libraries


def UFL_to_csc(a,f,bcs):
    
    """

    Assembles a weak form in UFL-form to a scipy sparse matrix and numpy array

    Inputs:

        a (UFL): UFL-form generated by dolfin
        f (UFL): UFL-form generated by dolfin

    Outputs:

        A (sps.csc_matrix): sparse matrix to be read by scipy
        F_mat (numpy.ndarray): numpy array 

    """
    
    A_PET = PETScMatrix()
    F_PET = PETScVector()
    
    assemble_system(a, f, bcs, A_tensor=A_PET, b_tensor=F_PET) 
    
    # convert to petsc4py object

    A_mat = A_PET.mat()
    
    F_mat = F_PET.get_local() 


    # convert to csr_matrix

    (A_indptr, A_ind, A_data) = A_mat.getValuesCSR()

    A = sps.csc_matrix((A_data, A_ind, A_indptr), shape=A_mat.getSize())

    A.eliminate_zeros()

 

    # return coo_matrix

    return A, F_mat

def UFL_to_csc_nobcs(a,f):
    
    """

    Assembles a weak form in UFL-form to a scipy sparse matrix and numpy array

    Inputs:

        a (UFL): UFL-form generated by dolfin
        f (UFL): UFL-form generated by dolfin

    Outputs:

        A (sps.csc_matrix): sparse matrix to be read by scipy
        F_mat (numpy.ndarray): numpy array 

    """
    
    A_PET = PETScMatrix()
    F_PET = PETScVector()
    
    assemble_system(a, f, A_tensor=A_PET, b_tensor=F_PET) 
    
    # convert to petsc4py object

    A_mat = A_PET.mat()
    
    F_mat = F_PET.get_local() 


    # convert to csr_matrix

    (A_indptr, A_ind, A_data) = A_mat.getValuesCSR()

    A = sps.csc_matrix((A_data, A_ind, A_indptr), shape=A_mat.getSize())

    A.eliminate_zeros()

    # return coo_matrix

    return A, F_mat

def POD(S,threshold,plot):
    """ 
    Performs a Proper Orthogonal Decomposition (POD) on a snapshot matrix

    Inputs: 

    S (numpy array) - snapshot matrix
    threshold (float) - cutoff value for the singular values
    plot (boolean) - plot for singular values

    Outputs : 

    V_r (numpy array) - transformation matrix
    s = (numpy array) - singular values

    """

    # SVD decomposition
    u, s, vh = np.linalg.svd(S, full_matrices=True)
    
    # Plot Singular Values
    if plot:
        xaxis = np.arange(1,len(s)+1)

        fig = plt.figure()
        ax = fig.add_subplot(2, 1, 1)
        ax.set_yscale('log')

        ax.plot(xaxis, s)
        ax.set_ylabel('Singular value')
        ax.set_xlabel('Index')

    # Define threshold so that we stop picking singular values below threshold

    for cutoff_index in range(0,len(s)):
        if s[cutoff_index] < threshold:
            break

    V_r = u[:,:cutoff_index+1]

    return V_r, s

def POD2(S,threshold : int ,plot):
    """ 
    Performs a Proper Orthogonal Decomposition (POD) on a snapshot matrix

    Inputs: 

    S (numpy array) - snapshot matrix
    threshold (integer) - cutoff value for the singular values
    plot (boolean) - plot for singular values

    Outputs : 

    V_r (numpy array) - transformation matrix
    s = (numpy array) - singular values

    """

    # SVD decomposition
    u, s, vh = np.linalg.svd(S, full_matrices=True)
    
    # Plot Singular Values
    if plot:
        xaxis = np.arange(1,len(s)+1)

        fig = plt.figure()
        ax = fig.add_subplot(2, 1, 1)
        ax.set_yscale('log')

        ax.plot(xaxis, s)
        ax.set_ylabel('Singular value')
        ax.set_xlabel('Index')
        plt.grid(True)
        plt.show()

    # Define threshold so that we stop picking singular values below threshold

    V_r = u[:,:threshold]

    return V_r, s

def POD_all(S1, SI, S2, cutoff1 : int, cutoffI : int, cutoff2 : int):
    """ 
    Performs a Proper Orthogonal Decomposition (POD) on a snapshot matrix

    Inputs: 

    S1, SI, S2 (numpy array) - snapshot matrices
    cutoff1, cutoffI, cutoff2 (integer) - cutoff values for the singular values

    Outputs : 
    
    V_1, V_I, V_2 (numpy array) - transformation matrices
    ndofs_u1, ndofs_uI, ndofs_u2 (integer) - DoFs in reduced spaces
    u1, uI, u2 (numpy array) - reduced-order snapshot matrices

    """
    
    S1_train = S1[:,0:nsamples-ntest]
    SI_train = SI[:,0:nsamples-ntest]
    S2_train = S2[:,0:nsamples-ntest]

    start = time.time()
    V_1, svs_1 = POD2(S1_train,cutoff1,0)
    V_I, svs_I = POD2(SI_train,cutoffI,0)
    end = time.time()
    
    POD1I_time = end-start
    
    start = time.time()
    V_2, svs_2 = POD2(S2_train,cutoff2,0)
    end = time.time()
    
    PODall_time = end-start+POD1I_time
    
    ndofs_u1 = np.shape(V_1)[1]
    ndofs_uI = np.shape(V_I)[1]
    ndofs_u2 = np.shape(V_2)[1]

    u1 = np.matmul(S1.T,V_1)
    uI = np.matmul(SI.T,V_I)
    u2 = np.matmul(S2.T,V_2)
    
    energy1 = sum(svs_1[0:cutoff1])/sum(svs_1)
    energy2 = sum(svs_2[0:cutoff2])/sum(svs_2)
    energyI = sum(svs_I[0:cutoffI])/sum(svs_I)
    
    print("Snapshot energy domain 1:",energy1)
    print("Snapshot energy domain 2:",energy2)
    print("Snapshot energy interface:",energyI)
    
    ## FIGURE ##
    
    xaxis1 = np.arange(1,len(svs_1)+1)
    xaxis2 = np.arange(1,len(svs_2)+1)
    xaxisI = np.arange(1,len(svs_I)+1)


    fig = plt.figure(figsize=(6.4, 4.8))
    plt.yscale('log')
    plt.xscale('log')

    plt.plot(xaxis1, svs_1, label='$\sigma_{\Omega_1, i}$', marker='o', linewidth = 2)
    plt.plot(xaxis2, svs_2, label='$\sigma_{\Omega_2, j}$', marker='o', linewidth = 2)
    plt.plot(xaxisI, svs_I, label='$\sigma_{\Gamma, k}$', marker='o', linewidth = 2)

    plt.legend()        
    plt.grid(True, which='major', linestyle='-')
    plt.grid(True, which='minor', linestyle='--')
    
    plt.savefig('svd', bbox_inches='tight')
    plt.show()
    
    return V_1, V_I, V_2, ndofs_u1, ndofs_uI, ndofs_u2, u1, uI, u2

def POD_M1(S1, SI, nsamples : int, ntest : int, cutoff1 : int, cutoffI : int):
    """
    Performs POD analysis on domain 1 and the interface
    
    Inputs: 

    S1, SI (numpy array) - snapshot matrices
    nsamples (integer) - number of samples
    ntest (integer) - number of test samples
    cutoff1, cutoffI (integer) - cutoff values for the singular values

    Outputs : 
    
    V_1, V_I (numpy array) - transformation matrices
    ndofs_u1, ndofs_uI (integer) - DoFs in reduced spaces
    PODtime (float) - computation time
    
    
    """
    
    start = time.time()
    
    S1_train = S1[:,0:nsamples-ntest]
    SI_train = SI[:,0:nsamples-ntest]

    V_1, svs_1 = POD2(S1_train,cutoff1,0)
    V_I, svs_I = POD2(SI_train,cutoffI,0)

    ndofs_u1 = np.shape(V_1)[1]
    ndofs_uI = np.shape(V_I)[1]

    end = time.time()

    PODtime = start - end
    
    print("We go from",ndofs_U1,"to",ndofs_u1,"modes on domain 1")
    print("We go from",ndofs_UI,"to",ndofs_uI,"modes on the interface")
    
    return V_1, V_I, ndofs_u1, ndofs_uI, PODtime

def plot_first_n_svalues(n,s):
    """
    Plots the first n singular values
    
    Inputs: 
    
    n (positive integer) - first n singular values
    s (numpy array) - singular values
    
    Outputs: 
    
    Figure of the first n singular values
    
    """
    xaxis = np.arange(1,n+1)
    fig = plt.figure()
    
    ax = fig.add_subplot(1,1,1)
    ax.plot(xaxis,s[0:n])
    ax.set_yscale('log')
    ax.set_ylabel('Singular value')
    ax.set_xlabel('Index')

def map_nodal_to_vertex(u,mesh):
    """
    
    Maps the nodal values to the vertex values
    
    Inputs: 
    
    u (numpy array) - nodal values
    
    Outputs: 
    
    u (numpy array) - vertex values
    
    """
    #V1 = FunctionSpace(mesh,"Lagrange", 1)
    mapping = vertex_to_dof_map(V)
    return u[mapping]

def block_vector_to_nodal(u_final,rows_domain1,rows_domain2,rows_interface,E_1,E_2,E_interface):
    
    """
    Rearranges the solution vector u = (u_1, u_2, u_\Gamma) to the nodal values
    
    Inputs:
    
    u_final (numpy array) - solution vector u
    
    rows_domain1 (numpy array) - contains the indices of the dofs on domain 1
    rows_domain2 (numpy array) - contains the indices of the dofs on domain 2
    rows_interface (numpy array) - contains the indices of the dofs on the interface
    
    E_1 (sps.csc_matrix ) - sparse matrix that rearranges the dofs on domain 1
    E_2 (sps.csc_matrix) - sparse matrix that rearranges the dofs on domain 2
    E_interface (sps.csc_matrix) - sparse matrix that rearranges the dofs on the interface
    
    Outputs
    
    v_final (numpy array) - solution vector as nodal values
    
    """

    u_part1 = u_final[:len(rows_domain1)]
    u_part2 = u_final[len(rows_domain1):len(rows_domain1)+len(rows_domain2)]
    u_partI = u_final[-len(rows_interface):]

    # Map back to nodal values
    v_final = E_1*u_part1 + E_2*u_part2 + E_interface*u_partI
    return v_final

def U_to_figure(U,rows_domain1,rows_domain2,rows_interface,E_1,E_2,E_interface,mesh,gridsize):
    
    """
    Creates a figure from the solution vector u = (u_1, u_2, u_\Gamma)
    
    Inputs:
    
    U (numpy array) - solution vector u
    
    rows_domain1 (numpy array) - contains the indices of the dofs on domain 1
    rows_domain2 (numpy array) - contains the indices of the dofs on domain 2
    rows_interface (numpy array) - contains the indices of the dofs on the interface
    
    E_1 (sps.csc_matrix ) - sparse matrix that rearranges the dofs on domain 1
    E_2 (sps.csc_matrix) - sparse matrix that rearranges the dofs on domain 2
    E_interface (sps.csc_matrix) - sparse matrix that rearranges the dofs on the interface
    
    mesh (dolfin.cpp.generation.UnitSquareMesh) - contains the mesh 
    gridsize (positive integer) - number of dofs per row on the mesh
    
    Outputs:
    
    v_final (numpy array) - solution vector as nodal values
    
    """

    U_nodal = block_vector_to_nodal(U,rows_domain1,rows_domain2,rows_interface,E_1,E_2,E_interface)
    U_vertex = map_nodal_to_vertex(U_nodal,mesh)
    u_reshape = np.reshape(U_vertex,(gridsize+1, gridsize+1))
    u_reshape = np.flipud(u_reshape)

    # Plot FOM
    plt.figure(figsize=(6.4, 4.8))
    plt.imshow(u_reshape, interpolation = "nearest",extent=[0,1,0,1], cmap='jet')
    plt.colorbar()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    
def block_to_fig(U,rows_domain1,rows_domain2,rows_interface,E_1,E_2,E_interface,mesh,gridsize):
    
    """
    Transforms the solution vector U from the block structure to an array to plot
    
    Inputs: 
    
    U (numpy array) - solution vector u
    
    rows_domain1 (numpy array) - contains the indices of the dofs on domain 1
    rows_domain2 (numpy array) - contains the indices of the dofs on domain 2
    rows_interface (numpy array) - contains the indices of the dofs on the interface
    
    E_1 (sps.csc_matrix ) - sparse matrix that rearranges the dofs on domain 1
    E_2 (sps.csc_matrix) - sparse matrix that rearranges the dofs on domain 2
    E_interface (sps.csc_matrix) - sparse matrix that rearranges the dofs on the interface
    
    mesh (dolfin.cpp.generation.UnitSquareMesh) - contains the mesh 
    gridsize (positive integer) - number of dofs per row on the mesh
    
    Outputs:
    
    u_reshape (numpy array) - array ready to plot
    
    """
    
    U_nodal = block_vector_to_nodal(U,rows_domain1,rows_domain2,rows_interface,E_1,E_2,E_interface)
    U_vertex = map_nodal_to_vertex(U_nodal,mesh)
    u_reshape = np.reshape(U_vertex,(gridsize+1, gridsize+1))
    
    u_reshape = np.flipud(u_reshape)
    
    return u_reshape
    
def nodal_to_figure(u, gridsize):
    
    """
    Creates a figure from the nodal values
    
    Inputs: 
    
    u_numpy (numpy array) - nodal values
    gridsize (integer) - size of the grid
    
    Outputs:
    
    Figure of the solution
    
    """
    # map from nodal to vertex values
    u_vertex = map_nodal_to_vertex(u, mesh)

    u_reshape = np.reshape(u_vertex,(gridsize+1, gridsize+1))
    u_reshape = np.flipud(u_reshape)

    # Plot
    plt.imshow(u_reshape, interpolation = "nearest",extent=[0,1,0,1])
    plt.colorbar()
    plt.show()
    
def split_data(data, train, val, test):

    """
    
    Splits the data set in train val en test set respectively
    
    Inputs: 
    
    data (numpy array) - data
    train (value between 0 and 1) - ratio of training data
    val (value between 0 and 1) - ratio of validation data
    test (value between 0 and 1) - ratio of test data
    
    Outputs: 
    
    train_data (numpy array) - training data
    val_data (numpy array) - validation data
    test_data (numpy array) - test data
    
    """
    
    train_len = int(np.floor(train*len(data)))
    val_len = int(np.floor(val*len(data)))
    test_len = int(np.floor(test*len(data)))

    train_data = data[:train_len]
    val_data = data[train_len:train_len+val_len]
    test_data = data[-test_len:]

    return train_data, val_data, test_data

def plot_last_epochs(n, train_losses_epochs, val_losses_epochs):
    
    """
    
    Plots the training and validation loss of the last n epochs
    
    Inputs:
    
    train_losses_epochs (numpy array) - array of training loss per epoch
    val_losses_epoch (numpy array) - array of validation loss per epoch
    n (positive integer) - number of last epochs to be shown
    
    Outputs:
    
    Figure of the last n epochs
    
    
    """ 
    xaxis = list(range(len(train_losses_epochs)-n, len(train_losses_epochs)))
    plt.plot(xaxis,train_losses_epochs[-n:], label='training loss')
    plt.plot(xaxis,val_losses_epochs[-n:], label='validation loss')
    plt.ylabel('loss')
    plt.legend()
    plt.xlabel('epoch')
    plt.show()
    
def relative_error(base, approximation):
    
    """
    Calculates the 2-norm relative error between base and approximation
    
    Inputs:
    
    base (numpy array) - array to compare with approximation
    approximation (numpy array) - approximation to base
    
    Outputs:
    
    rel_error (float) - relative error between base and approximation
    
    """
    
    rel_error = my_2norm(approximation-base)/my_2norm(base)
    
    return rel_error
    

def my_2norm(v):
    
    """
    
    Calculates the Euclidean norm of a vector
    
    Inputs:
    
    v (numpy array) - vector
    
    Outputs:
    
    (float) - Euclidean norm of v
    
    
    """
    mynorm = 0
    for i in range(len(v)):
        val = (v[i])**2
        mynorm = mynorm + val
        
    return sqrt(mynorm)


def get_inputs(mu_inputs, I_inputs, normalized):
    """
    Gives the inputs for the POD-NN on domain 2
    
    Inputs:
    
    mu_inputs (numpy array) - array of parameters
    I_inputs (numpy array) - array of interface values
    normalized (boolean) - normalized inputs yes or no
    
    Outputs:
    
    torch_inputs (torch) - inputs for the neural network
    if normalized:
        maxvalues_in (numpy array) - stores the max values per array
        minvalues_in (numpy array) - stores the min values per array
    
    """
    inputs = np.hstack([I_inputs, mu_inputs])
    torch_inputs = torch.from_numpy(inputs).float()
    
    if normalized:
        normalized_inputs = np.zeros((np.shape(inputs)[0],np.shape(inputs)[1]))
        maxvalues_in = np.zeros(np.shape(inputs)[1])
        minvalues_in = np.zeros(np.shape(inputs)[1])

        for i in range(np.shape(inputs)[1]):
            nvector, maxval, minval = normalize(inputs[:,i])
            normalized_inputs[:,i] = nvector
            maxvalues_in[i] = maxval
            minvalues_in[i] = minval
            
        torch_inputs = torch.from_numpy(normalized_inputs).float()
        
        return torch_inputs, maxvalues_in, minvalues_in
        
    else:
        return torch_inputs
            
    
def get_outputs(s_2,V_2, normalized):
    """
    Gives the outputs for the POD-NN on domain 2
    
    Inputs:
    
    s_2 (numpy array) - snapshot matrix of hybrid model on domain 2
    V_2 (numpy array) - transformation matrix of domain 2
    normalized (boolean) - normalized inputs yes or no
    
    Outputs:
      
    torch_outputs (torch) - torch of the outputs of the neural network
    if normalized:
        maxvalues_in (numpy array) - stores the max values per array
        minvalues_in (numpy array) - stores the min values per array
    
    """
    outputs = np.matmul(s_2.T,V_2)
    torch_outputs = torch.from_numpy(outputs).float()
    
    if normalized:
        normalized_outputs = np.zeros((np.shape(outputs)[0],np.shape(outputs)[1]))
        maxvalues_out = np.zeros(np.shape(outputs)[1])
        minvalues_out = np.zeros(np.shape(outputs)[1])

        for i in range(np.shape(outputs)[1]):
            nvector, maxval, minval = normalize(outputs[:,i])
            normalized_outputs[:,i] = nvector    
            maxvalues_out[i] = maxval
            minvalues_out[i] = minval
            
        torch_outputs = torch.from_numpy(normalized_outputs).float()
        
        return torch_outputs, maxvalues_out, minvalues_out
    
    else:
        return torch_outputs
    


def input_to_output_model_nonorm(u_I,mu,model):
    """
    For the neural networks, giving the inputs we get the output
    
    Inputs:
    u_I (numpy array) - array of interface values
    mu (numpy array) - array of parameter values
    model (torch.model) - corresponding neural network
    
    Outputs:
    output (numpy array) - prediction
    """
    
    inputs = np.hstack((u_I,mu))
    output = model(torch.from_numpy(inputs).float()).detach().numpy()  
    
    return output


def find_initial_condition(mu, mu_test, u_I_list, nsamples, ntest):
    """
    Finds the initial condition u_I depending on the closest mu
    
    Inputs:
    mu (numpy array) - array of parameter locations
    mu_test (numpy array) - parameter location of interest
    u_I_list (numpy array) - array of reduced interface vectors
    nsamples (int) - number of samples
    ntest (int) - number of test samples
    
    Outputs:
    closest_u_I (numpy array) - initial condition
    mu_norm (float) - 2-norm between closest mu and mu_test
    mu[index] - parameter location corresponding to initial condition
    
    """
    
    closest = np.linalg.norm(mu_test)
    for i in range(nsamples-ntest):
        candidate = np.linalg.norm(mu_test-mu[i])
        if candidate < closest:
            closest = candidate
            closest_u_I = u_I_list[i]
            index = i
            
    mu_norm = np.linalg.norm(mu[index]-mu_test)
            
    return closest_u_I, mu_norm, mu[index]



def newdescent(x_k, mu, h, gamma, a_tilde, f_tilde, threshold, traction_model):
    """
    Solves the non-linear equation in the online stage with unnormalized neural networks using gradient descent
        
    Inputs:
    x_k (numpy array) - initial condition
    mu (numpy array) - array of parameter values
    h (float) - used for finite differences
    a_tilde (numpy array) - input matrix in non-linear equation
    f_tilde (numpy array) - input vector in non-linear equation
    threshold (float) - threshold for the stopping criterion
    
    Outputs:
    x_k (numpy array) - solution vector
    delta_norm (float) - l2-norm between the last two iterations
    F_k (numpy array) - vector F in the last iteration
    count (integer) - number of iterations
    residuals (numpy array) - array of residuals 
    """
        
    count = 0
    
    residuals = []
        
    delta_norm = 1+threshold
    
    while delta_norm > threshold:
        
            
        t_hat_k = input_to_output_model_nonorm(x_k,mu,traction_model)

        # F_k for F_k(x_k)        
        F_k = np.matmul(a_tilde,x_k) + f_tilde - t_hat_k
        residuals.append(np.linalg.norm(F_k))

        # initialize Jacobian in step k by J_k
        J_k = np.ones((len(F_k),len(F_k)))

        for i in range(len(F_k)):

            # Determine x+h input
            x_kh = copy.deepcopy(x_k)

            x_kh[i] = x_k[i]+h

            t_hat_kh = input_to_output_model_nonorm(x_kh,mu,traction_model)

            value = (t_hat_kh-t_hat_k)/h
            J_k[:,i] = value
        
        J_k = a_tilde - J_k
        
        # Start of Backtracking Line Search to find a suitable step size
        
        # Define search control parameters
        
        c = 0.5
        tau = 0.5
        a = 10

        # Compute the gradient
        gradG = np.matmul(J_k.T,F_k)
        
        # Compute some parameters
        m = -np.linalg.norm(np.matmul(gradG,gradG))
        t = -c*m
        
        x_temp = x_k-a*gradG
        t_hat_temp = input_to_output_model_nonorm(x_temp,mu,traction_model)
        F_temp = np.matmul(a_tilde,x_temp) + f_tilde - t_hat_temp
        
        while np.linalg.norm(F_k)-np.linalg.norm(F_temp) < a*t:
            
            # update
            a = tau*a
            x_temp = x_k-a*gradG
            
            t_hat_temp = input_to_output_model_nonorm(x_temp,mu,traction_model)
            F_temp = np.matmul(a_tilde,x_temp) + f_tilde - t_hat_temp
            
        # Return step size gamma
        gamma = a
        
        # End of Backtracking line search
        
        # Determine x_k1
        x_k1 = x_k - gamma*gradG
        
        delta_norm = np.linalg.norm(x_k-x_k1)
        
        x_k = x_k1
        count = count + 1
        
        
        if delta_norm > 10e6:
            return 'No convergence', 'No convergence', 'No convergence', 'No convergence', 'No convergence'

    return x_k, delta_norm, F_k, count, residuals


def newton_nonorm(x_k, mu, h, a_tilde, f_tilde, criterion, threshold, traction_model):
    """
    Solves the non-linear equation in the online stage with unnormalized neural networks using newton's method
        
    Inputs:
    x_k (numpy array) - initial condition
    mu (numpy array) - array of parameter values
    h (float) - used for finite differences
    a_tilde (numpy array) - input matrix in non-linear equation
    f_tilde (numpy array) - input vector in non-linear equation
    criterion (boolean) - 1 is means Newton's algorithm stops when difference between the last two computed solutions
                          is smaller than the threshold
                          0 means Newton's algorithm stops when F(x_k) is smaller than the threshold
    threshold (float) - threshold for the criterion
    
    Outputs:
    x_k (numpy array) - solution vector
    delta_norm (float) - l2-norm between the last two iterations
    F_k (numpy array) - vector F in the last iteration
    count (integer) - number of iterations
    """
    
    count = 0
    residuals = []
    
    t_hat_k = input_to_output_model_nonorm(x_k,mu,traction_model)

    # F_k for F_k(x_k)
    F_k = np.matmul(a_tilde,x_k) + f_tilde - t_hat_k
    
    if criterion:
        delta_norm = np.linalg.norm(delta)
    else: 
        delta_norm = np.linalg.norm(F_k)
        
    while delta_norm > threshold:

        t_hat_k = input_to_output_model_nonorm(x_k,mu,traction_model)

        # F_k for F_k(x_k)
        F_k = np.matmul(a_tilde,x_k) + f_tilde - t_hat_k

        # initialize Jacobian in step k by J_k
        J_k = np.ones((len(F_k),len(F_k)))

        for i in range(len(F_k)):

            # Determine x+h input
            x_kh = copy.deepcopy(x_k)

            x_kh[i] = x_k[i]+h

            t_hat_kh = input_to_output_model_nonorm(x_kh,mu,traction_model)

            # Get F_kh
            F_kh = np.matmul(a_tilde,x_kh) + f_tilde - t_hat_kh

            value = (F_kh-F_k)/h
            J_k[:,i] = value


        # Determine x_k1-xk
        delta = np.linalg.solve(J_k,-F_k)
        
        if criterion:
            delta_norm = np.linalg.norm(delta)
        else: 
            delta_norm = np.linalg.norm(F_k)
            residuals.append(delta_norm)

            
        x_k1 = delta+x_k

        # update
        x_k = x_k1
        count = count + 1

        if delta_norm > 10e6:
            print('No convergence')
            
            if count > 1000:
                return x_k, delta_norm, F_k, 1000, residuals[0:1000]
            else: 
                return x_k, delta_norm, F_k, count, residuals

    return x_k, delta_norm, F_k, count, residuals

def Neural_Network(torch_inputs, torch_outputs, n_epochs, learning_rate):
    
    """
    Constructs and trains a neural network
    
    Inputs:
    
    torch_inputs (torch) - inputs for the neural network
    torch_outputs (torch) - outputs fo the neural network
    n_epochs (integer) - number of epochs
    learning_rate (float) - learning rate
    
    Outputs:
    
    model (sequential) - neural network architecture
    NNtime (float) - running time
    test_loss (float) - test loss
    
    """

    start = time.time()

    # Split data

    train = 0.8
    val = 0.1
    test = 0.1

    train_data_x, val_data_x, test_data_x = split_data(torch_inputs,train,val,test)
    train_data_y, val_data_y, test_data_y = split_data(torch_outputs,train,val,test)

    n_input = np.shape(train_data_x)[1]
    n_hidden = 50
    n_out = np.shape(train_data_y)[1]

    # Define structure of the NN
    model = nn.Sequential(nn.Linear(n_input, n_hidden),
                          nn.LeakyReLU(),
                          nn.Linear(n_hidden,n_hidden),
                          nn.LeakyReLU(),
                          nn.Linear(n_hidden,n_hidden),
                          nn.LeakyReLU(),
                          nn.Linear(n_hidden,n_hidden),
                          nn.LeakyReLU(),
                          nn.Linear(n_hidden, n_out))

    batch_size = 16

    # Define the loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    x = train_data_x
    y = train_data_y

    model = train_nn(model, train_data_x, train_data_y, val_data_x, val_data_y, n_epochs, batch_size, loss_function, optimizer)

    end = time.time()
    NNtime = end-start

    # Determine test loss
    model.eval()
    y_pred = model(test_data_x)

    test_loss = loss_function(y_pred, test_data_y) 

    y_predn = model(test_data_x).detach().numpy()
    test_data_yn = test_data_y.detach().numpy()

    avgrel = 0
    
    for i in range(len(y_pred)):
        error = relative_error(y_predn[i],test_data_yn[i])
        avgrel = avgrel + error
        
    
    return model, NNtime, test_loss

def Neural_Network_small(torch_inputs, torch_outputs, n_epochs, learning_rate):
    
    """
    Constructs and trains a small neural network
    
    Inputs:
    
    torch_inputs (torch) - inputs for the neural network
    torch_outputs (torch) - outputs fo the neural network
    n_epochs (integer) - number of epochs
    learning_rate (float) - learning rate
    
    Outputs:
    
    model (sequential) - neural network architecture
    NNtime (float) - running time
    test_loss (float) - test loss
    
    """

    start = time.time()

    # Split data

    train = 0.8
    val = 0.1
    test = 0.1

    train_data_x, val_data_x, test_data_x = split_data(torch_inputs,train,val,test)
    train_data_y, val_data_y, test_data_y = split_data(torch_outputs,train,val,test)

    n_input = np.shape(train_data_x)[1]
    n_hidden = 20
    n_out = np.shape(train_data_y)[1]

    # Define structure of the NN
    model = nn.Sequential(nn.Linear(n_input, n_hidden),
                          nn.LeakyReLU(),
                          nn.Linear(n_hidden,n_hidden),
                          nn.LeakyReLU(),
                          nn.Linear(n_hidden, n_out))

    batch_size = 16

    # Define the loss function and optimizer
    loss_function = nn.MSELoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    x = train_data_x
    y = train_data_y

    model = train_nn(model, train_data_x, train_data_y, val_data_x, val_data_y, n_epochs, batch_size, loss_function, optimizer)

    end = time.time()
    NNtime = end-start

    # Determine test loss
    model.eval()
    y_pred = model(test_data_x)

    test_loss = loss_function(y_pred, test_data_y) 

    y_predn = model(test_data_x).detach().numpy()
    test_data_yn = test_data_y.detach().numpy()

    avgrel = 0
    
    for i in range(len(y_pred)):
        error = relative_error(y_predn[i],test_data_yn[i])
        avgrel = avgrel + error
            
    return model, NNtime, test_loss.detach().numpy()

def train_nn(model, x, y, val_data_x, val_data_y, n_epochs, batch_size, lossf, opt):
    
    """
    Trains a neural network with validation data and early stopping algorithm
    
    Inputs:
    
    model (sequential) - neural network architecture
    x (torch) - input training data
    y (torch) - output training data
    val_data_x (torch) - input validation data
    val_data_y (torch) - output validation data
    n_epochs (integer) - number of epochs
    batch_size (integer) - batch size
    lossf (function) - loss function
    opt (function) - optimization function
  
    Outputs:
    
    best model (function) - returns neural network
    
    """
    
    # For early stopping
    val_loss_0 = 1e10
    
    n_batches = int(len(x) / batch_size)

    # Initialize loss lists
    train_losses_steps = []
    train_losses_epochs = []
    
    val_losses_epochs = []

    for epoch in range(n_epochs):
        
        # Initialize epoch loss
        epochloss = 0
        
        print("Current epoch: "+ str(epoch+1), end="\r")

        # Training loop
        for batch in range(n_batches):
            
            # Create batches
            batch_X, batch_y = x[batch*batch_size:(batch+1)*batch_size,], y[batch*batch_size:(batch+1)*batch_size,]

            # Make prediction on the batch and save the loss
            pred_y = model(batch_X)
            loss = lossf(pred_y, batch_y)

            # Add loss to the array and sum the loss to calculate loss per epoch
            train_losses_steps.append(loss.item())
            epochloss += loss

            # Backward propagation
            model.zero_grad()
            loss.backward() 

            # Update weights and biases
            opt.step()

      # Validation loop
        with torch.no_grad(): 
            model.eval() 

            pred_y = model(val_data_x) 
            val_loss = lossf(pred_y, val_data_y) 
            val_losses_epochs.append(val_loss)
            
            # Early stopping
            if val_loss < val_loss_0:
                
                best_model = model
                best_epoch = epoch
                val_loss_0 = val_loss

        # Append average epoch loss
        train_losses_epochs.append(epochloss.item()/n_batches)

    print("\n Best epoch:",best_epoch)

    # Plot loss
#     plt.plot(train_losses_epochs, label='training loss')
#     plt.plot(val_losses_epochs, label='validation loss')

#     plt.ylabel('MSE loss')
#     plt.legend()
#     plt.xlabel('epoch')
#     plt.show()
    
    return best_model

def BFGS_Fu(x,mu_test,a1_tilde,f1_tilde, traction_model):
    
    """
    Computes the function value of F(u_\Gamma) as in (3.18)
    
    Inputs:
    
    x (numpy array) - reduced-interface vector
    mu_test (numpy array) - parameter location
    a1_tilde (numpy array) - a^{(1)}_{ΓΓ} − a_{Γ1}a^{−1}_{11} a_{1Γ}
    f1_tilde (numpy array) - a_{Γ1}a^{−1}_{11} f_1 − f^{(1)}_Γ
    traction_model (mapping) - mapping of the traction model

    Outputs:
    
    (numpy array) Function value at x
    
    """
    
    t_hat = input_to_output_model_nonorm(x,mu_test,traction_model)
    
    return np.matmul(a1_tilde,x) + f1_tilde - t_hat 

def BFGS_g(x, mu_test, a1_tilde, f1_tilde, traction_model):
    
    """
    Objective function as in (3.26) for BFGS algorithm
    
    Inputs:
    
    x (numpy array) - reduced interface vector
    mu_test (numpy array) - parameter location
    a1_tilde (numpy array) - a^{(1)}_{ΓΓ} − a_{Γ1}a^{−1}_{11} a_{1Γ}
    f1_tilde (numpy array) - a_{Γ1}a^{−1}_{11} f_1 − f^{(1)}_Γ
    traction_model (mapping) - mapping of the traction model
    
    Outputs:
    
    g(x) (numpy array) - objective
    
    """

    return 0.5*np.dot(BFGS_Fu(x,mu_test,a1_tilde,f1_tilde, traction_model).T,BFGS_Fu(x,mu_test,a1_tilde,f1_tilde,traction_model))

def BFGS_grad_g(x, h, mu_test, a1_tilde, f1_tilde, traction_model):
    
    """
    Computes the gradient at x using first-order finite differences for the BFGS algorithm
    
    Inputs: 
    
    x (numpy array) - location to compute the gradient
    h (float) - parameter for finite differences
    h (float) - parameter for finite differences
    mu_test (numpy array) - parameter location
    a1_tilde (numpy array) - a^{(1)}_{ΓΓ} − a_{Γ1}a^{−1}_{11} a_{1Γ}
    f1_tilde (numpy array) - a_{Γ1}a^{−1}_{11} f_1 − f^{(1)}_Γ
    traction_model (mapping) - mapping of the traction model
    
    Outputs:
    
    grad (numpy array) - gradient at x
    
    """

    
    # Initialize
    grad = np.zeros((len(x)))
    
    # Loop through arguments of g
    for i in range(len(x)):
        
        x_h = copy.deepcopy(x)  
        
        # Calculate the derivative using finite differences
        x_h[i] = x[i]+h
        val = (BFGS_g(x_h, mu_test, a1_tilde, f1_tilde, traction_model)-BFGS_g(x, mu_test, a1_tilde, f1_tilde, traction_model))/h
        
        # Store
        grad[i] = val
    
    return grad

def BFGS_line_search(c, tau, a, x, p, h, mu_test, a1_tilde, f1_tilde, traction_model):
    
    """
    Performs the backtracking line search algorithm for the BFGS algorithm
    
    Inputs:
    
    c, tau (float) - control parameters
    a (float) - initial step size
    x (numpy array) - location 
    p (numpy array) - descent direction
    h (float) - parameter for finite differences
    mu_test (numpy array) - parameter location
    a1_tilde (numpy array) - a^{(1)}_{ΓΓ} − a_{Γ1}a^{−1}_{11} a_{1Γ}
    f1_tilde (numpy array) - a_{Γ1}a^{−1}_{11} f_1 − f^{(1)}_Γ
    traction_model (mapping) - mapping of the traction model
    
    Outputs:
    
    a (float) - optimized step size
    
    """
    
    # Compute some parameters
    m = np.matmul(BFGS_grad_g(x, h, mu_test, a1_tilde, f1_tilde, traction_model),p)
    t = -c*m

    while BFGS_g(x, mu_test, a1_tilde, f1_tilde, traction_model) - BFGS_g(x+a*p, mu_test, a1_tilde, f1_tilde, traction_model) < a*t:

        # update
        a = tau*a

    # Return step size
    return a   

def BFGS(x_k, H_k, threshold, h, mu_test, a1_tilde, f1_tilde, traction_model):
    """
    Performs the BFGS algorithm with stopping criterion based on the norm of the gradient of G
    
    Inputs:
    
    x_k (numpy array) - reduced interface vector
    H_k (numpy array) - Initial guess Hessian
    threshold (float) - threshold for norm of grad(G)
    h (float) - parameter for finite differences
    mu_test (numpy array) - parameter location
    a1_tilde (numpy array) - a^{(1)}_{ΓΓ} − a_{Γ1}a^{−1}_{11} a_{1Γ}
    f1_tilde (numpy array) - a_{Γ1}a^{−1}_{11} f_1 − f^{(1)}_Γ
    traction_model (mapping) - mapping of the traction model
    
    Outputs: 
    
    x_k (numpy array) - reduced interface vector
    residuals (numpy array) - residuals per iteration
    count (integer) - number of iterations
    
    """

    count = 0
    delta_norm = 1+threshold
    residuals = []

    while delta_norm > threshold:

        residual = np.linalg.norm(BFGS_Fu(x_k,mu_test,a1_tilde,f1_tilde, traction_model))

        # Determine Gradient at x_k
        grad_G = BFGS_grad_g(x_k, h, mu_test, a1_tilde, f1_tilde, traction_model)

        # Determine search direction
        p_k = np.matmul(-H_k, grad_G)

        # Perform back-tracking line search
        lr = BFGS_line_search(0.5, 0.5, 10, x_k, p_k, h, mu_test, a1_tilde, f1_tilde, traction_model)

        s_k = lr*p_k
        x_kp1 = x_k+s_k


        # Determine Gradient at x_kp1
        grad_G_h = BFGS_grad_g(x_kp1,h, mu_test, a1_tilde, f1_tilde, traction_model)

        y_k = grad_G_h-grad_G

        # Approximate the Hessian
        rho_k = 1/np.dot(y_k.T,s_k)
        H_kp1 = np.dot(np.dot(np.identity(len(x_k))-np.dot(np.dot(s_k,rho_k),y_k.T),H_k),np.identity(len(x_k))-np.dot(np.dot(rho_k,y_k),s_k.T))+np.dot(np.dot(rho_k,s_k),s_k.T)

        # Update
        H_k = H_kp1 
        delta_norm = np.linalg.norm(y_k) 

        x_k = x_kp1
        count = count + 1

        # Store
        residuals.append(residual)
    
    return x_k, residuals, count

def Fu(x,mu_test,a1_tilde,f1_tilde):
    
    """
    Computes the function value of F(u_\Gamma) as in (3.18)
    
    Inputs:
    
    x (numpy array) - reduced-interface vector
    mu_test (numpy array) - parameter location
    a1_tilde (numpy array) - a^{(1)}_{ΓΓ} − a_{Γ1}a^{−1}_{11} a_{1Γ}
    f1_tilde (numpy array) - a_{Γ1}a^{−1}_{11} f_1 − f^{(1)}_Γ
    
    Outputs:
    
    (numpy array) Function value at x
    
    """
    
    t_hat = input_to_output_model_nonorm(x,mu_test,traction_model)
    
    return np.matmul(a1_tilde,x) + f1_tilde - t_hat 

def g(x):
    
    """
    Objective function as in (3.26)
    
    Inputs:
    
    x (numpy array) - reduced interface vector
    
    Outputs:
    
    g(x) (numpy array) - objective
    
    """

    return 0.5*np.dot(Fu(x,mu_test,a1_tilde,f1_tilde).T,Fu(x,mu_test,a1_tilde,f1_tilde))

def grad_g(x, h):
    
    """
    Computes the gradient at x using first-order finite differences 
    
    Inputs: 
    
    x (numpy array) - location to compute the gradient
    h (float) - parameter for finite differences
    
    Outputs:
    
    grad (numpy array) - gradient at x
    
    """
    
    # Initialize]
    grad = np.zeros((len(x)))
    
    # Loop through arguments of g
    for i in range(len(x)):
        
        x_h = copy.deepcopy(x)  
        
        # Calculate the derivative using finite differences
        x_h[i] = x[i]+h
        val = (g(x_h)-g(x))/h
        
        # Store
        grad[i] = val
    
    return grad

def line_search(c, tau, a, x, p):
    
    """
    Performs the backtracking line search algorithm
    
    Inputs:
    
    c, tau (float) - control parameters
    a (float) - initial step size
    x (numpy array) - location 
    p (numpy array) - descent direction
    
    Outputs:
    
    a (float) - optimized step size
    
    """
    
    # Compute some parameters
    m = np.matmul(grad_g(x,h),p)
    t = -c*m

    while g(x) - g(x+a*p) < a*t:

        # update
        a = tau*a

    # Return step size
    return a  


def standardize(data):
    
    """
    Standardizes data
    
    Inputs: 
    
    data (numpy array) - data to standardize
    
    Outputs:
    
    standardized_data (numpy array) - standardized data
    
    """
    
    standardized_data = np.zeros((np.shape(data)))
    
    means = np.mean(data,axis=0)
    stds = np.std(data,axis=0)
    
    for idx, row in enumerate(data):
        standardized_row = (row-means)/stds
        standardized_data[idx] = standardized_row
        
    return standardized_data

def normalize(data):
    
    """
    Normalizes data
    
    Inputs: 
    
    data (numpy array) - data to normalize
    
    Outputs:
    
    normalized_data (numpy array) - normalized data
    maxs (numpy array) - array of max values (to denormalize later)
    mins (numpy array) - array of min values (to denormalize later)
    
    """
    
    normalized_data = np.zeros((np.shape(data)))
    
    maxs = np.max(data,axis=0)
    mins = np.min(data,axis=0)
    
    for idx, row in enumerate(data):
        normalized_row = (row - mins)/(maxs - mins)
        normalized_data[idx] = normalized_row
        
    return normalized_data, maxs, mins

def Monte_Carlo(OoI_array, nsamples, mean : bool):
    
    """
    Performs Monte Carlo simulation for mean or variance
    
    Inputs:
    
    OoI_array (numpy array) - array of outputs of interest
    nsamples (integer) - number of samples
    mean (boolean) - 1 for mean, 0 for variance
    
    Outputs
    
    QoI_iter (numpy array) - quantity of interest per iteration
    QoI_confidence_95_low_iter (numpy array) - lower bound confidence interval of QoI per iteration
    QoI_confidence_95_high_iter (numpy array) - upper bound confidence interval of QoI per iteration
    
    """
    
    QoI_iter = np.zeros((nsamples-ntest))
    QoI_confidence_95_low_iter = np.zeros((nsamples))
    QoI_confidence_95_high_iter = np.zeros((nsamples))
    
    variance = np.var(OoI_array)
    
    for i in range(nsamples):
            if mean:
                QoI_iter[i] = np.mean(OoI_array[0:i+1])
            else: 
                QoI_iter[i] = np.var(OoI_array[0:i+1])

            QoI_confidence_95_low = QoI_iter[i]-1.96*sqrt(variance/(i+1)) 
            QoI_confidence_95_high = QoI_iter[i]+1.96*sqrt(variance/(i+1))

            QoI_confidence_95_low_iter[i] = QoI_confidence_95_low
            QoI_confidence_95_high_iter[i] = QoI_confidence_95_high
            
    # Figure 
    
    xaxis = np.arange(100,nsamples-ntest)
    fig = plt.plot(xaxis,QoI_iter[100:])
    plt.plot(xaxis,QoI_confidence_95_low_iter[100:])
    
    plt.plot(xaxis,QoI_confidence_95_high_iter[100:])
    plt.xlabel('Number of samples')
    legend = plt.legend(['Quantity of Interest','Lower bound 95%-CI','Upper bound 95%-CI'])
    
    plt.show()
    
    return QoI_iter, QoI_confidence_95_low_iter, QoI_confidence_95_high_iter

def f_normal(x,mu,sigma):
    """
    Probability density function of the normal distribution
    
    Inputs:
    
    x (float) - field variable
    mu (float) - mean
    sigma (float) - standard deviation
    
    Outputs: 
    
    PDF value at x
    
    """
    return (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x-mu)/sigma)**2)

def uni_dist(x, a , b):
    
    """
    Probability density function of the uniform distribution
    
    Inputs:
    
    x (float) - field variable
    a, b (float) - left and right interval values
    
    Outputs: 
    
    prob_density (float) - PDF value at x
    
    """
    if x >= a and x <= b:
        prob_density = 1/(b-a)
    else:
        prob_density = 0
    return prob_density